In [11]:
import os
import sys
import glob
import json
import tqdm
import shutil

import numpy as np
import pandas as pd
import nibabel as nib

sys.path.append(os.path.join("..", "..", ".."))
from tta_uia_segmentation.src.preprocessing.utils import get_filepaths_raw

## Extract the bias corrected TOF scans

In [18]:
output_dir = '../../../../data/preprocessed/0_bias_corrected/Lausanne'

scans_dict = get_filepaths_raw(
    path_to_tof_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/N4_bias_field_corrected',
    fp_pattern_tof=['*', '*', '*', '*angio_N4bfc_mask.nii.gz'],
    path_to_seg_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/skull_stripped_and_aneurysm_mask',
    fp_pattern_seg=['*', '*', '*', '*Lesion_1_mask.nii.gz'],
    level_of_dir_with_id=-4,
    every_scan_has_seg=False    
)

In [19]:
for scan_id, scans in tqdm.tqdm(scans_dict.items()):
    scan_dir_path = os.path.join(output_dir, scan_id)
    os.makedirs(scan_dir_path, exist_ok=True)
    
    # copy new_path_tof to output_dir_tof
    new_path_tof = os.path.join(scan_dir_path, f'{scan_id}_tof.nii.gz')
    shutil.copy(scans['tof'], new_path_tof)

    # copy new_path_aneurysm to output_dir_label
    if 'seg' in scans:
        new_path_aneurysm = os.path.join(scan_dir_path, f'{scan_id}_seg.nii.gz')
        shutil.copy(scans['seg'], new_path_aneurysm)
    


100%|██████████| 38/38 [00:17<00:00,  2.21it/s]


In [16]:
scans

{'tof': '/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/N4_bias_field_corrected/sub-482/ses-20140210/anat/sub-482_ses-20140210_desc-angio_N4bfc_mask.nii.gz'}